In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/next-word-prediction/1661-0.txt


In [2]:
file_loc = '/kaggle/input/next-word-prediction/1661-0.txt'
with open(file_loc, encoding='utf8') as f:
    text= f.readlines()
len(text)

12310

In [3]:
text = open(file_loc, encoding='utf8').read().lower()
print('corpus length:', len(text))

corpus length: 581888


In [4]:
text = text.lower()

Importing important modeuls and classes

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

2024-08-07 08:44:06.714186: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 08:44:06.714290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 08:44:06.808164: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
tokenizer = Tokenizer()

In [7]:
tokenizer.fit_on_texts([text])

In [8]:
# Checking all the words and their indices
# tokenizer.word_index
len(tokenizer.word_index)

8931

In [9]:
inp_seq = []
for sent in text.split('\n'):
    if sent:
        tokenized_sent = tokenizer.texts_to_sequences([sent])[0]
    for i in range(1, len(tokenized_sent)):
        inp_seq.append(tokenized_sent[:i+1])
        

In [10]:
max_len = max([len(x) for x in inp_seq])

### Now Adding the 0-padding

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

pad_inp = pad_sequences(inp_seq, maxlen = max_len, padding = 'pre')

In [12]:
pad_inp

array([[   0,    0,    0, ...,    0,  145, 4790],
       [   0,    0,    0, ...,  145, 4790,    1],
       [   0,    0,    0, ..., 4790,    1, 1020],
       ...,
       [   0,    0,    0, ...,    3,  360,   83],
       [   0,    0,    0, ...,  360,   83,  358],
       [   0,    0,    0, ...,   83,  358, 1673]], dtype=int32)

This is in the format where we can say the last ele is the output and the rest are the input

In [13]:
x = pad_inp[:, : -1]
y = pad_inp[:, -1]

x.shape, y.shape

((118152, 19), (118152,))

### Making Categrical Columns

In [14]:
from tensorflow.keras.utils import to_categorical

In [15]:
total_words = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes= total_words)

In [16]:
y.shape

(118152, 8932)

### Making models

In [23]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [24]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_len - 1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))

model.build(input_shape=(None, max_len - 1))

In [25]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [26]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 19, 100)        │       893,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8932)           │     1,152,228 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,162,676 (8.25 MB)

 Trainable params: 2,162,676 (8.25 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
x.shape

(118152, 19)

In [28]:
history = model.fit(x, y, epochs = 100)

Epoch 1/100
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 28s 7ms/step - accuracy: 0.0588 - loss: 6.5194
Epoch 2/100
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.1323 - loss: 5.4341
Epoch 3/100
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.1657 - loss: 4.9384
Epoch 4/100
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.1893 - loss: 4.5564
Epoch 5/100
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - accuracy: 0.2149 - loss: 4.2129
Epoch 6/100
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - accuracy: 0.2457 - loss: 3.8884
Epoch 7/100
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - accuracy: 0.2791 - loss: 3.6022
Epoch 8/100
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - accuracy: 0.3171 - loss: 3.3240
Epoch 9/100
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - accuracy: 0.3568 - loss: 3.0767
Epoch 10/100
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - accuracy: 0.3955 - loss: 2.8585
Epoch 11/100
3693/3693 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - accuracy: 0.4303 - loss: 2.6540
Epoch 12

In [29]:
model.save('NextWordModel.h5')

In [131]:
tokenizer_json = tokenizer.to_json()
with open('/kaggle/working/WordTokenizer.json', 'w') as f:
    f.write(tokenizer_json)

### Testing the model

In [124]:
test_text = "random"

In [125]:
test_token = tokenizer.texts_to_sequences([test_text])[0]
test_pad = pad_sequences([test_token], maxlen = max_len, padding = 'pre')

In [126]:
test_pad

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0, 8369]],
      dtype=int32)

#### Predection

In [127]:
pre = model.predict(test_pad)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [128]:
ans_idx = np.argmax(pre)

In [129]:
tokenizer.index_word[ans_idx]

'each'

getting top 5 words

In [130]:
top_idx = np.argsort(pre)[0][-5:][::-1]
for i in top_idx:
    print(test_text, " ",tokenizer.index_word[i])

random   each
random   swinging
random   corners
random   mat
random   blood


(0.024242105, 0.6656224, 0.16164017)

In [97]:
# predecting next 5 words
for i in range(10):
    test_token = tokenizer.texts_to_sequences([test_text])[0]
    test_pad = pad_sequences([test_token], maxlen = max_len, padding = 'pre')
    
    pre = model.predict(test_pad)
    ans_word = np.argmax(pre)
    res_word = tokenizer.index_word[ans_word]
    print(res_word)
    
    test_text = test_text + " " + res_word
    
test_text

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
each
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
other
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
over
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
between
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
to
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
us
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
was
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
speeding


'random each other over and between to us and was speeding'